In [3]:
import xarray as xr
import numpy as np
import cftime
import nc_time_axis
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
#from cftime import datetime 
from datetime import datetime, timedelta
#import datetime as dt
import matplotlib.colors as colors
import math
import random
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator
#jet = mcm.get_cmap('jet')
jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')
import netCDF4 as nc
import sys
import os
import psutil
import netCDF4 as nc
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from dateutil import tz
import pytz
import time
from time import process_time
from tqdm import tqdm
from dask.distributed import LocalCluster, Client
import dask.array as da
import dask.dataframe as dd
import dask
from dask import delayed
from dask.diagnostics import ProgressBar  # Import the ProgressBar class
import calendar
import glob
%matplotlib inline 
#import line_profiler
#%load_ext line_profiler

/tmp/ipykernel_616535/1870643686.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')


In [6]:
fp1 = 'Nc_Files/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h2.0001-*.nc'
ds = xr.open_mfdataset(fp1)
#ds

<xarray.Dataset>
Dimensions:       (lat: 96, lon: 144, time: 8760, mlat: 97, mlon: 80, lev: 126,
                   ilev: 127, nbnd: 2)
Coordinates:
  * lat           (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon           (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * mlat          (mlat) float64 -90.0 -88.12 -86.24 -84.33 ... 86.24 88.12 90.0
  * mlon          (mlon) float64 -180.0 -175.5 -171.0 ... 166.5 171.0 175.5
  * lev           (lev) float64 4.055e-10 5.207e-10 6.686e-10 ... 970.6 992.6
  * ilev          (ilev) float64 3.551e-10 4.559e-10 5.854e-10 ... 985.1 1e+03
  * time          (time) object 0001-01-01 00:00:00 ... 0001-12-31 23:00:00
Dimensions without coordinates: nbnd
Data variables: (12/113)
    gw            (time, lat) float64 dask.array<chunksize=(24, 96), meta=np.ndarray>
    hyam          (time, lev) float64 dask.array<chunksize=(24, 126), meta=np.ndarray>
    hybm          (time, lev) float64 dask.array<chunksize=(24, 126), meta=np.ndarray>
    P0            (time) float64 1e+05 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
    hyai          (time, ilev) float64 dask.array<chunksize=(24, 127), meta=np.ndarray>
    hybi          (time, ilev) float64 dask.array<chunksize=(24, 127), meta=np.ndarray>
    ...            ...
    UI            (time, lev, lat, lon) float32 dask.array<chunksize=(24, 126, 96, 144), meta=np.ndarray>
    V             (time, lev, lat, lon) float32 dask.array<chunksize=(24, 126, 96, 144), meta=np.ndarray>
    VI            (time, lev, lat, lon) float32 dask.array<chunksize=(24, 126, 96, 144), meta=np.ndarray>
    WI            (time, lev, lat, lon) float32 dask.array<chunksize=(24, 126, 96, 144), meta=np.ndarray>
    Z3            (time, lev, lat, lon) float32 dask.array<chunksize=(24, 126, 96, 144), meta=np.ndarray>
    e             (time, lev, lat, lon) float32 dask.array<chunksize=(24, 126, 96, 144), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport
    logname:           earfw
    host:              d13s7b4.arc4.lee
    initial_file:      /nobackup/earfw/cesm2/cases/ACP_CESM213_FX2000_f19_f19...
    topography_file:   /nobackup/earfw/cesm2_inputdata//atm/cam/topo/USGS-gto...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  hour_1

In [16]:
timee = ds.variables['time']

lon = ds.variables['lon']
lat = ds.variables['lat']
lev = ds.variables['lev']
dst = ds.transpose("lev", ...)
#geopH = dst.variables['Z3'] / 1000 #m-> km    #(126, 744, 96, 144)        #Mpt = dst.variables['Fep'] 

temp = dst.variables['T']

Fept = dst.variables['Fep']
Mgpt = dst.variables['Mgp']
Napt = dst.variables['Nap']


lev_sl_idx_min = 41      
lev_sl_idx_max = 60 

lev_sl = lev[lev_sl_idx_min:lev_sl_idx_max+1]

tempe = temp[lev_sl_idx_min:lev_sl_idx_max+1,:,:,:]

Fep_t = Fept[lev_sl_idx_min:lev_sl_idx_max+1,:,:,:]
Mgp_t = Mgpt[lev_sl_idx_min:lev_sl_idx_max+1,:,:,:]
Nap_t = Napt[lev_sl_idx_min:lev_sl_idx_max+1,:,:,:]


Feptdens = ( Fep_t * 1e-6 * 100 * lev_sl ) / (1.380503e-23 * tempe)
Mgptdens = ( Mgp_t * 1e-6 * 100 * lev_sl ) / (1.380503e-23 * tempe)
Naptdens = ( Nap_t * 1e-6 * 100 * lev_sl ) / (1.380503e-23 * tempe)
            
Mptdens = Feptdens + (2 * Mgptdens ) + Naptdens    # equivalent to [Fep] * Factor where Factor = (Fep+2Mgp+Nap)/Fep

In [19]:
tempe.shape #(20, 8760, 96, 144)

(20, 8760, 96, 144)

In [27]:
len(lev_sl)
range(len(lev_sl))

range(0, 20)

In [ ]:
#print( f'Mptdens min = {Mptdens.min().values}' )



min_values = []

for lev_idx in tqdm( range(len(lev_sl)) ):
    for lat_idx in range(len(lat)):
        for lon_idx in range(len(lon)):
            for time_idx in range(len(timee)):
                Fept_val = Fept[lev_idx, time_idx, lat_idx, lon_idx]
                Mgp_val = Mgpt[lev_idx, time_idx, lat_idx, lon_idx]
                Nap_val = Napt[lev_idx, time_idx, lat_idx, lon_idx]
                tempe_val = tempe[lev_idx, time_idx, lat_idx, lon_idx]

                Feptdens_val = (Fept_val * 1e-6 * 100 * lev[lev_idx]) / (1.380503e-23 * tempe_val)
                Mgpdens_val = (Mgp_val * 1e-6 * 100 * lev[lev_idx]) / (1.380503e-23 * tempe_val)
                Naptdens_val = (Nap_val * 1e-6 * 100 * lev[lev_idx]) / (1.380503e-23 * tempe_val)
                
                Mptdens_val = Feptdens_val + (2 * Mgpdens_val) + Naptdens_val

                min_values.append(Mptdens_val)

min_value = np.min(min_values)
print(f'Mptdens min = {min_value}')





In [ ]:
print( f'Mptdens max = {Mptdens.max().values}' )